<a href="https://colab.research.google.com/github/Kavyapriyakp/Program-GUI/blob/main/Moving_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisite**
---

**Cloning the repository**

In [1]:
%cd /content/
!git clone https://github.com/Kavyapriyakp/Program-GUI.git
%cd /content/Program-GUI
input_offset = 'demo/'

/content
Cloning into 'Program-GUI'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 347 (delta 33), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (347/347), 65.80 MiB | 31.02 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/Program-GUI


**Installing dependencies**

In [2]:
!pip install numpy
!sudo apt-get install python-opencv
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  python-opencv
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 535 kB of archives.
After this operation, 2,944 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Fetched 535 kB in 1s (756 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package python-opencv.
(Reading database ..

**Importing libraries**

In [3]:
import numpy as np
import cv2
from PIL import ImageTk, Image
import cv2
from cv2 import *
import numpy as np
import sys
import time
import imutils

# **Browsing video input**


In [13]:
from google.colab import files
import os
import shutil

input_user_path = os.path.join(input_offset, 'input_user')

if os.path.isdir(input_user_path):
    shutil.rmtree(input_user_path)
os.mkdir(input_user_path)

uploaded = files.upload()

for key in uploaded.keys():
    abspath = os.path.abspath(key)
    os.rename(abspath, os.path.join(input_user_path, key))

Saving Video2.mp4 to Video2.mp4


# **Parameters**
---
Set Distance of Coverage (DoC) and Time Delay (TD) for improved performance.





In [5]:
Distance_of_Coverage = 8  #@param {type: "slider", min: 0, max: 10}
Time_Delay = 0  #@param {type: "slider", min: 0, max: 10}

**(Optional) Locating the input**

In [16]:
filepath=os.path.join(input_user_path, key)
print(filepath)

demo/input_user/Video2.mp4


# **(Optional) Displaying the input**
This module supports all video file formats.  However, only .MP4 files can be displayed on Google Colaboratory. 
If the input video is of any other format, it will not be displayed here.  In that case, kindly skip this step.

In [17]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(filepath,'rb').read()
data_url = f"data:{filepath};base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4" autoplay="True">
</video>
""" % data_url)

# **Main Program**

In [10]:
from google.colab.patches import cv2_imshow

kernel_d = np.ones((3,3), np.uint8)
kernel_e = np.ones((3,3), np.uint8)
kernel_gauss = (3,3)
dilate_times = 13                       #initializing_integer_variables
erode_times = 5                         #initializing_integer_variables
is_blur = True                          #initializing_boolean_variables
is_close = True                         #initializing_boolean_variables
is_draw_ct = False                      #initializing_boolean_variables
fac = 2                                 #initializing_integer_variables
def drawRectangle(frame, minus_frame):
	if(is_blur):
		minus_frame = GaussianBlur(minus_frame, kernel_gauss, 0)
	minus_Matrix = np.float32(minus_frame)	
	if(is_close):
		for i in range(dilate_times):
			minus_Matrix = dilate(minus_Matrix, kernel_d)
		
		for i in range(erode_times):
			minus_Matrix = erode(minus_Matrix, kernel_e)
		
	minus_Matrix = np.clip(minus_Matrix, 0, 255)
	minus_Matrix = np.array(minus_Matrix, np.uint8)
	contours, hierarchy = findContours(minus_Matrix.copy(), RETR_TREE, CHAIN_APPROX_SIMPLE)
	for c in contours:
		(x, y, w, h) = boundingRect(c)	
		rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
		if( is_draw_ct ):
			drawContours(frame, contours, -1, (0, 255, 255), 2)
	cv2_imshow(frame)
	!ffmpeg -y -loglevel panic -i filepath output.mp4
def objdetect():
	capture = cv2.VideoCapture(filepath);
	while(1):
		(ret_old, old_frame) = capture.read()
		
		gray_oldframe = cvtColor(old_frame, COLOR_BGR2GRAY)
		if(is_blur):
			gray_oldframe = GaussianBlur(gray_oldframe, kernel_gauss, 0)
		oldBlurMatrix = np.float32(gray_oldframe)
		accumulateWeighted(gray_oldframe, oldBlurMatrix, 0.003)
		while(True):
			ret, frame = capture.read()	
			
			gray_frame = cvtColor(frame, COLOR_BGR2GRAY)
			if(is_blur):
				newBlur_frame = GaussianBlur(gray_frame, kernel_gauss, 0)
			else:
				newBlur_frame = gray_frame
			newBlurMatrix = np.float32(newBlur_frame)
			minusMatrix = absdiff(newBlurMatrix, oldBlurMatrix)
			ret, minus_frame = threshold(minusMatrix, 60, 255.0, THRESH_BINARY)
			accumulateWeighted(newBlurMatrix,oldBlurMatrix,0.02)
			
			drawRectangle(frame, minus_frame)
		capture.release() 
		cv2.destroyAllWindows()

# **Moving Object Detection**

In [ ]:
objdetect()

*End of Program*

---









